In [1]:
import pandas as pd
from tsai.all import SlidingWindowPanel
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [2]:
from importlib.machinery import SourceFileLoader
SourceFileLoader(
    'tcn', r'/content/tcn/tcn.py').load_module()

<module 'tcn' from '/content/tcn/tcn.py'>

In [3]:
def norm_df(df,cols):
  for col in cols:
    df[col] = df[col] / df[col].max()
  return df

In [7]:
df_shift7 = pd.read_pickle('merge_df_shift_7.pickle')
df_shift7 = df_shift7[['date', 'NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'name', 'tavg',  'prcp',
       'NDVI_mean_shift', 'tavg_shift', 'prcp_shift']]
df_shift14 = pd.read_pickle('merge_df_shift_14.pickle')
df_shift14 = df_shift14[['date', 'NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'name', 'tavg',
        'prcp','NDVI_mean_shift', 'tavg_shift', 'prcp_shift']]

In [8]:
df_shift7 = norm_df(df_shift7, ['tavg','prcp','tavg_shift','prcp_shift'])
df_shift14 = norm_df(df_shift14, ['tavg','prcp','tavg_shift','prcp_shift'])
df_shift7.dropna(axis = 0, inplace = True)
df_shift14.dropna(axis = 0, inplace = True)

In [9]:
df_shift14.columns

Index(['date', 'NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'name', 'tavg', 'prcp',
       'NDVI_mean_shift', 'tavg_shift', 'prcp_shift'],
      dtype='object')

In [10]:
x_cols = ['NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'tavg_shift', 'prcp_shift']

In [11]:
sliding_window_X, sliding_window_y = SlidingWindowPanel(window_len=14, unique_id_cols=[
                                                        'name'], get_x=x_cols, get_y=['NDVI_mean_shift'], stride=1)(df_shift14)


processing data...


...data processed
concatenating X...
...X concatenated
concatenating y...
...y concatenated


In [12]:
sliding_window_X.shape

(1139006, 5, 14)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(sliding_window_X, sliding_window_y,
                                                    random_state=42, shuffle=False, test_size=0.2)

In [14]:
X_train.shape

(911204, 5, 14)

In [15]:
X_train = np.moveaxis(X_train, 0, 1)
X_train = np.moveaxis(X_train, 0, -1)
X_train = np.moveaxis(X_train, 0, 1)
X_train.shape

(14, 911204, 5)

In [16]:
X_test = np.moveaxis(X_test, 0, 1)
X_test = np.moveaxis(X_test, 0, -1)
X_test = np.moveaxis(X_test, 0, 1)
X_test.shape

(14, 227802, 5)

In [17]:
import tensorflow as tf
from tcn import TCN, compiled_tcn
tf.config.list_physical_devices()


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [18]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
model = compiled_tcn(num_feat=X_train.shape[-1], num_classes=1, nb_filters=64,
                     kernel_size=6, dilations=[1, 2, 4, 8, 16, 32, 64], nb_stacks=2, max_len=911204)


In [17]:
y_train.shape

(911204,)

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=5, validation_split=0.1)

Epoch 1/5


In [19]:
def compile_model(swp_X):

    # output tcn2
    nb_features = swp_X.shape[2]

    i = Input(shape=(swp_X.shape[1], nb_features))

    m = TCN(nb_filters=64, kernel_size=3, dropout_rate=0.6, dilations=[
            1, 2, 4, 8, 16, 32], return_sequences=True, activation='relu')(i)
    m = TCN(nb_filters=32, dropout_rate=0.6, kernel_size=3, dilations=[
            1, 2, 4, 8, 16, 32], return_sequences=False, activation='relu')(m)
    m = Dense(1, activation='linear')(m)
    model = Model(inputs=[i], outputs=[m])

    model.summary()
    opt = tf.keras.optimizers.Adam()
    model.compile(loss=root_mean_squared_error, optimizer=opt,
                  metrics=['mae', root_mean_squared_error])
    return model

In [ ]:
model = compile_model(X_train)
history = model.fit(X_train, y_train, epochs=100, batch_size=5, validation_split=.1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 911204, 5)]       0         
                                                                 
 tcn (TCN)                   (None, 911204, 64)        169408    
                                                                 
 tcn_1 (TCN)                 (None, 32)                44608     
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 214,049
Trainable params: 214,049
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
